In [84]:
import numpy as np
import shutil
import os

In [85]:
buffer_size = 10000
total_size = 55000

### Helper functions

In [86]:
def save_array_to_file(file_name, array_to_save):
    np.savetxt(file_name, array_to_save, fmt = '%d')
    
def sort_and_write(file_name, array_to_sort):
    mergeSort(array_to_sort, 0, len(array_to_sort)-1)
    save_array_to_file(file_name, array_to_sort)
    
def read_n_int(file_, numbers_to_read):
    array_ = []
    
    if numbers_to_read <= 0 :
        return array_
    
    num = file_.readline()
    while(num):
        array_.append(int(num))
        if len(array_) >= numbers_to_read:
            break
        num = file_.readline()
            
    return array_

def create_unsorted_file(size_, file_name_ = 'unsorted.csv'):
    arr = np.arange(size_)
    np.random.shuffle(arr)
    save_array_to_file(file_name_, arr)
    arr = None

## External Sort

### Merge Sort

In [87]:
def merge(arr, l, m, r):
    n1 = m - l + 1
    n2 = r - m
 
    L = [0] * (n1)
    R = [0] * (n2)
 
    for i in range(0, n1):
        L[i] = arr[l + i]
 
    for j in range(0, n2):
        R[j] = arr[m + 1 + j]
 
    i = 0     
    j = 0     
    k = l     
 
    while i < n1 and j < n2:
        if L[i] <= R[j]:
            arr[k] = L[i]
            i += 1
        else:
            arr[k] = R[j]
            j += 1
        k += 1
 
    while i < n1:
        arr[k] = L[i]
        i += 1
        k += 1
 
    while j < n2:
        arr[k] = R[j]
        j += 1
        k += 1
 
 
 
def mergeSort(arr, l, r):
    if l < r:
 

        m = l+(r-l)//2
 
        mergeSort(arr, l, m)
        mergeSort(arr, m+1, r)
        merge(arr, l, m, r)

In [88]:
import heapq

def sort_slices(file_name, buffer_size_):
    read_arr = []
    chunk = 1
    f = open(file_name)

    if os.path.exists('./tmp/'):
        shutil.rmtree('./tmp/')
    os.mkdir('./tmp/')

    read_arr = read_n_int(f, buffer_size_)
    while (len(read_arr) > 0):
        sort_and_write('./tmp/sorted_' + str(chunk), read_arr)
        read_arr = read_n_int(f, buffer_size_)
        chunk = chunk + 1

    f.close()

def min_heap_sort(output_file):
    sorted_file = open(output_file, 'w+')

    min_heap = []
    heapq.heapify(min_heap)
    
    open_files = []
    for f in os.listdir('./tmp/'):
        if os.path.isfile('./tmp/' + f):
            file_ = open('./tmp/' + f)
            open_files.append(file_)
            val = file_.readline()
            heapq.heappush(min_heap, (int(val), file_))

    while(len(min_heap) > 0):
        min_element = heapq.heappop(min_heap)
        sorted_file.write(str(min_element[0]) + '\n')
        next_str = min_element[1].readline()
        if next_str:
            heapq.heappush(min_heap, (int(next_str), min_element[1]))
        else:
            min_element[1].close()

    sorted_file.close()

In [89]:
def external_sort(input_file, output_file, buffer_size_ = 10000):
    sort_slices(input_file, buffer_size_)
    min_heap_sort(output_file)
    print('Sorted values are written to' , str(output_file))

In [90]:
%%time
create_unsorted_file(total_size, file_name_ = 'unsorted.csv')

CPU times: user 109 ms, sys: 2.9 ms, total: 112 ms
Wall time: 114 ms


In [91]:
import pandas as pd
unsorted=pd.read_csv("/content/unsorted.csv")
unsorted.head(10)

,4991
0,7470
1,41523
2,36645
3,21051
4,27738
5,10320
6,317
7,39171
8,35109
9,10314


## This function performs external sort, just call it with input and output file name and it will do the magic

In [92]:
%%time
external_sort(input_file= 'unsorted.csv', output_file='sorted_external.csv', buffer_size_ = buffer_size)

Sorted values are written to sorted_external.csv
CPU times: user 648 ms, sys: 14.1 ms, total: 663 ms
Wall time: 665 ms


In [93]:
sorted=pd.read_csv("/content/sorted_external.csv")
sorted.head(10)

,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


### Checking Temp Files

In [94]:
if(total_size%buffer_size==0):
  n=total_size//buffer_size
else:
  n=total_size//buffer_size
  n=n+1
for i in range(1,n+1):
  print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~","File Number",i,"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  display(pd.read_csv("/content/tmp/sorted_"+str(i),index_col=0).head(10))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ File Number 1 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


""
2
6
13
17
19
22
28
32
35
36


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ File Number 2 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


""
0
8
10
11
30
39
40
47
56
70


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ File Number 3 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


...

""
7
9
14
16
21
23
24
26
33
43


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ File Number 5 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


""
1
3
15
18
29
31
34
38
42
51


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ File Number 6 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


""
5
25
58
61
66
76
99
101
103
107


### Delete tmp 

In [95]:
shutil.rmtree("/content/tmp")